# Identity & Access Management (IAM) Overview

Identity and Access Management is the security discipline that ensures the **right individuals** access the **right resources** at the **right times** for the **right reasons**.

```
┌─────────────────────────────────────────────────────────────────────────┐
│                        IAM Core Pillars                                 │
├─────────────────┬─────────────────┬─────────────────┬──────────────────┤
│  IDENTIFICATION │ AUTHENTICATION  │  AUTHORIZATION  │   ACCOUNTABILITY │
│  "Who are you?" │ "Prove it"      │ "What can you   │  "What did you   │
│                 │                 │   access?"      │    do?"          │
├─────────────────┼─────────────────┼─────────────────┼──────────────────┤
│  • Username     │ • Passwords     │ • RBAC          │  • Audit Logs    │
│  • Email        │ • MFA/2FA       │ • ABAC          │  • Session Track │
│  • Employee ID  │ • Biometrics    │ • ReBAC         │  • Compliance    │
│  • Certificates │ • Tokens        │ • Policies      │  • Forensics     │
└─────────────────┴─────────────────┴─────────────────┴──────────────────┘
```

## 1. Authentication Methods

Authentication verifies that users are who they claim to be using one or more **factors**:

| Factor Type | Description | Examples |
|-------------|-------------|----------|
| **Something You Know** | Knowledge-based | Passwords, PINs, security questions |
| **Something You Have** | Possession-based | Hardware tokens, smartphones, smart cards |
| **Something You Are** | Inherence-based | Fingerprints, facial recognition, iris scan |
| **Somewhere You Are** | Location-based | GPS, IP geolocation, network location |
| **Something You Do** | Behavior-based | Typing patterns, mouse movements, gait |

### Multi-Factor Authentication (MFA) Flow

```
┌──────────┐      ┌───────────────────────────────────────────────────────┐
│   USER   │      │                  AUTH SERVER                          │
└────┬─────┘      └───────────────────────┬───────────────────────────────┘
     │                                    │
     │  1. Username + Password            │
     │───────────────────────────────────>│
     │                                    │  ✓ Verify Factor 1
     │  2. Request Second Factor          │
     │<───────────────────────────────────│
     │                                    │
     │  3. TOTP Code / Push Confirm       │
     │───────────────────────────────────>│
     │                                    │  ✓ Verify Factor 2
     │  4. Access Token Issued            │
     │<───────────────────────────────────│
     │                                    │
```

### Password Security Best Practices
- **Minimum 12+ characters** with complexity requirements
- Use **password managers** for unique passwords per service
- Implement **rate limiting** and account lockout policies
- Store using **Argon2id, bcrypt, or scrypt** (never MD5/SHA1)
- Check against **breached password databases** (HaveIBeenPwned)

## 2. Authorization Models

Authorization determines what authenticated users are permitted to do.

### Role-Based Access Control (RBAC)

```
┌─────────────────────────────────────────────────────────────────┐
│                         RBAC Model                              │
│                                                                 │
│   USERS ──────> ROLES ──────> PERMISSIONS ──────> RESOURCES    │
│                                                                 │
│  ┌───────┐    ┌─────────┐    ┌────────────┐    ┌────────────┐  │
│  │ Alice │───>│  Admin  │───>│ read,write │───>│ Database   │  │
│  │ Bob   │───>│  Editor │───>│ delete     │───>│ Files      │  │
│  │ Carol │───>│  Viewer │───>│ execute    │───>│ APIs       │  │
│  └───────┘    └─────────┘    └────────────┘    └────────────┘  │
└─────────────────────────────────────────────────────────────────┘
```

**Pros:** Simple, auditable, widely supported  
**Cons:** Role explosion, coarse-grained, static

---

### Attribute-Based Access Control (ABAC)

```
┌──────────────────────────────────────────────────────────────────────────┐
│                            ABAC Policy Engine                            │
│                                                                          │
│  Subject Attrs    +  Resource Attrs  +  Environment   =   Decision      │
│  ┌────────────┐      ┌────────────┐    ┌───────────┐     ┌──────────┐   │
│  │ role=eng   │      │ type=doc   │    │ time=9AM  │     │  ALLOW   │   │
│  │ dept=IT    │  +   │ class=conf │  + │ ip=corp   │ =>  │    or    │   │
│  │ clearance  │      │ owner=X    │    │ device=   │     │  DENY    │   │
│  └────────────┘      └────────────┘    └───────────┘     └──────────┘   │
└──────────────────────────────────────────────────────────────────────────┘

Example Policy: "Engineers can read confidential docs during business hours from corp network"
```

**Pros:** Fine-grained, dynamic, context-aware  
**Cons:** Complex policies, harder to audit, performance overhead

---

### Relationship-Based Access Control (ReBAC)

```
┌─────────────────────────────────────────────────────────────────┐
│                    ReBAC Graph Model                            │
│                                                                 │
│     ┌───────┐  owner   ┌────────┐  parent  ┌────────────┐      │
│     │ Alice │─────────>│ Folder │─────────>│ Org: Acme  │      │
│     └───────┘          └────┬───┘          └─────┬──────┘      │
│                             │ contains           │ member      │
│                             v                    v             │
│                        ┌────────┐           ┌───────┐          │
│                        │ Doc.pdf│           │  Bob  │          │
│                        └────────┘           └───────┘          │
│                                                                 │
│  Query: Can Bob view Doc.pdf?                                  │
│  Answer: Yes (Bob is member of Acme, which is parent of Folder)│
└─────────────────────────────────────────────────────────────────┘
```

**Used by:** Google Zanzibar, Auth0 FGA, SpiceDB, Ory Keto

## 3. OAuth 2.0 & OpenID Connect (OIDC)

**OAuth 2.0** = Authorization framework (delegated access)  
**OIDC** = Identity layer on top of OAuth 2.0 (authentication)

### OAuth 2.0 Authorization Code Flow (with PKCE)

```
┌──────────┐                              ┌───────────────┐                    ┌────────────┐
│  User    │                              │   Client App  │                    │ Auth Server│
│ (Browser)│                              │               │                    │            │
└────┬─────┘                              └───────┬───────┘                    └──────┬─────┘
     │  1. Click "Login"                         │                                   │
     │──────────────────────────────────────────>│                                   │
     │                                           │                                   │
     │  2. Redirect to Auth Server               │                                   │
     │     (+ code_challenge, state, scope)      │                                   │
     │<──────────────────────────────────────────│                                   │
     │                                           │                                   │
     │  3. Login + Consent                       │                                   │
     │──────────────────────────────────────────────────────────────────────────────>│
     │                                           │                                   │
     │  4. Redirect back with Authorization Code │                                   │
     │<──────────────────────────────────────────────────────────────────────────────│
     │                                           │                                   │
     │  5. Forward code to Client                │                                   │
     │──────────────────────────────────────────>│                                   │
     │                                           │                                   │
     │                                           │  6. Exchange code + code_verifier │
     │                                           │     for tokens (back-channel)     │
     │                                           │──────────────────────────────────>│
     │                                           │                                   │
     │                                           │  7. Access Token + ID Token       │
     │                                           │<──────────────────────────────────│
     │  8. Authenticated Session                 │                                   │
     │<──────────────────────────────────────────│                                   │
```

### Key OAuth 2.0 Grant Types

| Grant Type | Use Case | Security |
|------------|----------|----------|
| **Authorization Code + PKCE** | Web apps, mobile, SPAs | ✅ Most secure |
| **Client Credentials** | Service-to-service (M2M) | ✅ Secure |
| **Device Code** | Smart TVs, CLI tools | ✅ Secure |
| **Implicit** | Legacy SPAs | ❌ Deprecated |
| **Password Grant** | Legacy, trusted apps only | ❌ Avoid |

### OIDC Tokens

```
┌─────────────────────────────────────────────────────────────────────────┐
│  ID Token (JWT)           │  Access Token            │  Refresh Token  │
├───────────────────────────┼──────────────────────────┼─────────────────┤
│  • User identity claims   │  • API authorization     │  • Get new      │
│  • iss, sub, aud, exp     │  • Short-lived (mins)    │    access token │
│  • name, email, picture   │  • Opaque or JWT         │  • Long-lived   │
│  • For client consumption │  • For resource server   │  • Secure store │
└───────────────────────────┴──────────────────────────┴─────────────────┘
```

## 4. SAML (Security Assertion Markup Language)

SAML is an **XML-based** standard for exchanging authentication/authorization data, commonly used in **enterprise SSO**.

```
┌──────────────────────────────────────────────────────────────────────────┐
│                    SAML SSO Flow (SP-Initiated)                          │
└──────────────────────────────────────────────────────────────────────────┘

┌──────────┐           ┌───────────────────┐           ┌───────────────────┐
│   User   │           │  Service Provider │           │ Identity Provider │
│ (Browser)│           │   (SP - Your App) │           │  (IdP - Okta/AD)  │
└────┬─────┘           └─────────┬─────────┘           └─────────┬─────────┘
     │  1. Access App            │                               │
     │──────────────────────────>│                               │
     │                           │                               │
     │  2. SAML AuthnRequest     │                               │
     │     (Redirect to IdP)     │                               │
     │<──────────────────────────│                               │
     │                           │                               │
     │  3. AuthnRequest          │                               │
     │──────────────────────────────────────────────────────────>│
     │                           │                               │
     │  4. User Authenticates    │                               │
     │   (if not already)        │                               │
     │<─────────────────────────────────────────────────────────>│
     │                           │                               │
     │  5. SAML Response (POST)  │                               │
     │     with signed Assertion │                               │
     │<──────────────────────────────────────────────────────────│
     │                           │                               │
     │  6. POST Assertion to SP  │                               │
     │──────────────────────────>│                               │
     │                           │  7. Validate Signature        │
     │                           │     Create Session            │
     │  8. Access Granted        │                               │
     │<──────────────────────────│                               │
```

### SAML vs OIDC Comparison

| Aspect | SAML 2.0 | OIDC |
|--------|----------|------|
| **Format** | XML | JSON/JWT |
| **Transport** | HTTP POST/Redirect | HTTP REST |
| **Token Size** | Large (KB) | Compact |
| **Best For** | Enterprise SSO | Web/Mobile/APIs |
| **Complexity** | High | Moderate |
| **Mobile Support** | Poor | Excellent |

## 5. Zero Trust Architecture

**"Never trust, always verify"** — Zero Trust assumes no implicit trust based on network location.

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                    Traditional Perimeter vs Zero Trust                      │
├─────────────────────────────────────┬───────────────────────────────────────┤
│         PERIMETER MODEL             │           ZERO TRUST MODEL            │
│                                     │                                       │
│    ┌─────────────────────────┐      │                                       │
│    │    TRUSTED NETWORK      │      │   ┌─────┐    ┌─────┐    ┌─────┐      │
│    │  ┌───┐ ┌───┐ ┌───┐     │      │   │ App │    │ App │    │ App │      │
│    │  │App│ │App│ │App│     │      │   │ + 🔒│    │ + 🔒│    │ + 🔒│      │
│    │  └───┘ └───┘ └───┘     │      │   └──┬──┘    └──┬──┘    └──┬──┘      │
│    │     Trust assumed      │      │      │          │          │          │
│    └────────────┬───────────┘      │      │    VERIFY EVERY     │          │
│         🔥 FIREWALL 🔥             │      │      REQUEST        │          │
│    ┌────────────┴───────────┐      │   ┌──┴──────────┴──────────┴──┐       │
│    │    UNTRUSTED INTERNET  │      │   │     Policy Engine (PDP)   │       │
│    └────────────────────────┘      │   └───────────────────────────┘       │
│                                     │                                       │
│   ❌ Castle-and-moat mentality     │   ✅ Every request authenticated      │
│   ❌ Flat internal network         │   ✅ Micro-segmentation               │
│   ❌ Implicit trust inside         │   ✅ Least privilege access           │
└─────────────────────────────────────┴───────────────────────────────────────┘
```

### Zero Trust Principles (NIST SP 800-207)

1. **All data sources and services are resources** — Protect everything
2. **All communication is secured** — Encrypt regardless of network
3. **Access is granted per-session** — No persistent trust
4. **Access is determined by dynamic policy** — Context-aware decisions
5. **Monitor and measure security posture** — Continuous assessment
6. **Authentication and authorization are dynamic** — Enforce before access
7. **Collect as much info as possible** — Improve security posture

### Zero Trust Components

```
┌────────────────────────────────────────────────────────────────┐
│                    Zero Trust Architecture                     │
│                                                                │
│  ┌─────────┐    ┌──────────────────────────────────────────┐  │
│  │ Subject │───>│ Policy Enforcement Point (PEP)           │  │
│  │ (User/  │    │ • Proxy / Gateway / Agent                │  │
│  │ Device) │    └────────────────┬─────────────────────────┘  │
│  └─────────┘                     │                            │
│                                  v                            │
│              ┌───────────────────────────────────────┐        │
│              │  Policy Decision Point (PDP)          │        │
│              │  • Evaluate: identity, device health, │        │
│              │    location, behavior, data class     │        │
│              └───────────────────────────────────────┘        │
│                           │                                   │
│    ┌──────────────────────┼───────────────────────────┐       │
│    v                      v                           v       │
│  ┌─────────┐      ┌──────────────┐            ┌───────────┐  │
│  │  SIEM   │      │ Threat Intel │            │ IAM / IdP │  │
│  └─────────┘      └──────────────┘            └───────────┘  │
└────────────────────────────────────────────────────────────────┘
```

## 6. Session Management

Session management maintains user state after authentication.

```
┌──────────────────────────────────────────────────────────────────────────┐
│                     Session Management Approaches                        │
├────────────────────────────┬─────────────────────────────────────────────┤
│    SERVER-SIDE SESSIONS    │         STATELESS TOKENS (JWT)             │
├────────────────────────────┼─────────────────────────────────────────────┤
│                            │                                             │
│  Client ───Cookie────> Server    Client ───JWT Header────> Server       │
│          (Session ID)     │              (Self-contained)               │
│               │           │                    │                        │
│               v           │                    v                        │
│         ┌─────────┐       │         ┌───────────────────┐               │
│         │ Session │       │         │ Validate signature│               │
│         │  Store  │       │         │ Check expiration  │               │
│         │(Redis/DB)│       │         │ Extract claims    │               │
│         └─────────┘       │         └───────────────────┘               │
│                            │                                             │
│  ✅ Easy revocation       │  ✅ Stateless, scalable                     │
│  ✅ Small cookie size     │  ✅ No server storage                       │
│  ❌ Requires shared store │  ❌ Hard to revoke                          │
│  ❌ Scaling complexity    │  ❌ Token size can grow                     │
└────────────────────────────┴─────────────────────────────────────────────┘
```

### Session Security Best Practices

| Practice | Implementation |
|----------|----------------|
| **Secure cookies** | `Secure; HttpOnly; SameSite=Strict` |
| **Session timeout** | Idle timeout (15-30 min), absolute timeout (8-24 hrs) |
| **Session regeneration** | New session ID after login (prevent fixation) |
| **Binding** | Bind to IP/User-Agent (with care for mobile) |
| **Revocation** | Blocklist for JWTs, delete from store for sessions |
| **Token refresh** | Short-lived access tokens + refresh token rotation |

## 7. Identity Federation

Federation enables users to access multiple systems with a **single identity** managed by a trusted provider.

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                        Identity Federation Architecture                     │
│                                                                             │
│    ┌──────────────────────────────────────────────────────────────────┐    │
│    │                    TRUST CIRCLE (Federation)                      │    │
│    │                                                                   │    │
│    │     ┌───────────┐      Trust       ┌───────────┐                 │    │
│    │     │   Corp    │<────────────────>│  Partner  │                 │    │
│    │     │   IdP     │    Agreement     │    IdP    │                 │    │
│    │     └─────┬─────┘                  └─────┬─────┘                 │    │
│    │           │                              │                       │    │
│    │    ┌──────┴──────┐              ┌────────┴───────┐               │    │
│    │    │  Internal   │              │   Partner      │               │    │
│    │    │   Users     │              │    Users       │               │    │
│    │    └──────┬──────┘              └────────┬───────┘               │    │
│    │           │                              │                       │    │
│    │           └──────────────┬───────────────┘                       │    │
│    │                          │                                       │    │
│    │                          v                                       │    │
│    │              ┌───────────────────────┐                           │    │
│    │              │    Service Provider   │                           │    │
│    │              │      (Your App)       │                           │    │
│    │              │  Accepts assertions   │                           │    │
│    │              │  from both IdPs       │                           │    │
│    │              └───────────────────────┘                           │    │
│    └───────────────────────────────────────────────────────────────────┘    │
└─────────────────────────────────────────────────────────────────────────────┘
```

### Federation Protocols & Standards

| Protocol | Use Case | Token Format |
|----------|----------|-------------|
| **SAML 2.0** | Enterprise SSO, B2B | XML Assertions |
| **OIDC** | Consumer/Modern apps | JWT (ID Token) |
| **WS-Federation** | Legacy Microsoft | XML |
| **SCIM** | User provisioning | JSON |

### Common Federation Patterns

```
┌─────────────────────────────────────────────────────────────────┐
│  1. ENTERPRISE SSO          2. SOCIAL LOGIN      3. B2B ACCESS │
│                                                                 │
│  ┌───────┐                  ┌─────────┐         ┌──────────┐   │
│  │ Okta/ │   Employee       │ Google/ │ Consumer│ Partner  │   │
│  │ Azure │   Identity       │ Facebook│ Identity│ Company  │   │
│  │  AD   │                  │ Apple   │         │   IdP    │   │
│  └───┬───┘                  └────┬────┘         └────┬─────┘   │
│      │                           │                   │         │
│      v                           v                   v         │
│  Internal Apps              Consumer App        Shared Portal  │
└─────────────────────────────────────────────────────────────────┘
```

### Just-In-Time (JIT) Provisioning

Automatically create/update user accounts during first federated login:

1. User authenticates at IdP
2. IdP sends assertion with user attributes
3. SP checks if user exists → if not, creates account
4. Maps IdP attributes to local user properties
5. Grants access based on federated identity

## 8. Quick Reference & Key Takeaways

### Security Checklist

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    IAM Security Checklist                               │
├─────────────────────────────────────────────────────────────────────────┤
│  □ Enforce MFA for all privileged accounts                             │
│  □ Implement least privilege access (RBAC/ABAC)                        │
│  □ Use OAuth 2.0 + PKCE for modern applications                        │
│  □ Set appropriate session timeouts and token lifetimes                │
│  □ Validate all tokens server-side (signature, expiry, audience)       │
│  □ Secure cookies: HttpOnly, Secure, SameSite=Strict                   │
│  □ Implement account lockout and rate limiting                         │
│  □ Log all authentication events for audit                             │
│  □ Regular access reviews and certification                            │
│  □ Plan for Zero Trust migration                                       │
└─────────────────────────────────────────────────────────────────────────┘
```

### Protocol Decision Tree

```
                        Need Identity/Auth?
                              │
              ┌───────────────┴───────────────┐
              │                               │
       Enterprise SSO?                 Modern App/API?
              │                               │
       ┌──────┴──────┐                 ┌──────┴──────┐
       │             │                 │             │
   Legacy WS-Fed   SAML 2.0         OAuth 2.0    OIDC
   (avoid if new)  (enterprise)    (authZ only)  (authN+authZ)
```

### Common Vulnerabilities to Avoid

| Vulnerability | Mitigation |
|---------------|------------|
| Credential stuffing | MFA, rate limiting, breached password checks |
| Session fixation | Regenerate session ID after auth |
| Token theft | Short expiry, secure storage, token binding |
| Privilege escalation | Validate authZ on every request |
| Insecure direct object refs | Check ownership before access |
| Open redirect | Whitelist redirect URIs |